# Performance comparaison

Delaney's aquaous solubility  

| Fingerprint | MLP R2| MLP RMSE| RF R2 |RF RMSE | 
|:-:|:-:|:-:| :-:| :-:|
|BERT on ChEMBL|0.842| - |0.797|-| 
|ECFP| 0.715 |- | 0.639 | -|
|Can2Can|0.642|-|0.618|-|
|Enum2Enum|0.676|-|0.640|-|
|Transformer|0.842|-|0.772|-|
|MPNN| 0.903 | 0.662 |-|-|
| Graph Convolution | 0.877 | 0.744|-|-|
| Weave |0.897 | 0.681|-|-|

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import torch
import optuna

from bert import BERT
from build_vocab import WordVocab
from utils import split

In [2]:
pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

In [3]:
df_train = pd.read_csv('data/ESOL_train.csv')
df_test = pd.read_csv('data/ESOL_test.csv')
df_train.head()

,SMILES,unknown,solubility,processed_smiles
0,C=CC1C2CCN(C1)C(C2)C(c1cc[nH0]c2ccc(cc12)OC)O,56-54-2,-3.37,C = C C 1 C 2 C C N ( C 1 ) C ( C 2 ) C ( c 1 ...
1,Brc1c(Br)cccc1,583-53-9,-3.50,Br c 1 c ( Br ) c c c c 1
2,c1c2cccc3c4c(cccc4)c(c23)cc1,206-44-0,-6.00,c 1 c 2 c c c c 3 c 4 c ( c c c c 4 ) c ( c 2 ...
3,c1c(C)c2Cc3c(cccc3)c2cc1,1730-37-6,-5.22,c 1 c ( C ) c 2 C c 3 c ( c c c c 3 ) c 2 c c 1
4,CCCC(C)(C)O,590-36-3,-0.49,C C C C ( C ) ( C ) O


In [7]:
x_train = [split(sm) for sm in df_train['SMILES']]
y_train = df_train['solubility']
x_test = [split(sm) for sm in df_test['SMILES']]
y_test = df_test['solubility']

In [8]:
vocab = WordVocab.load_vocab('data/vocab.pkl')

In [9]:
def get_inputs(sm):
    seq_len = 220
    ids = [vocab.stoi.get(token, unk_index) for token in sm.split()]
    ids = [sos_index] + ids + [eos_index] + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    
    return ids, seg

In [10]:
def get_array(x):
    x_id, x_seg = [], []
    for sm in x:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)
    
xid_train, xseg_train = get_array(x_train)
xid_test, xseg_test = get_array(x_test)

In [11]:
print(xid_test.shape)
print(xseg_test.shape)

torch.Size([129, 220])
torch.Size([129, 220])


# Encode to fingerprint

In [31]:
# model = BERT(len(vocab), hidden=256, n_layers=4, attn_heads=4, dropout=0)
# model.load_state_dict(torch.load('../result/GDB17/ep00_it020000.pkl'))
# model.eval()

BERT(
  (embedding): BERTEmbedding(
    (token): TokenEmbedding(45, 256, padding_idx=0)
    (position): PositionalEmbedding()
    (segment): SegmentEmbedding(3, 256, padding_idx=0)
    (dropout): Dropout(p=0)
  )
  (transformer_blocks): ModuleList(
    (0): TransformerBlock(
      (attention): MultiHeadedAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): Linear(in_features=256, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (output_linear): Linear(in_features=256, out_features=256, bias=True)
        (attention): Attention()
        (dropout): Dropout(p=0)
      )
      (feed_forward): PositionwiseFeedForward(
        (w_1): Linear(in_features=256, out_features=1024, bias=True)
        (w_2): Linear(in_features=1024, out_features=256, bias=True)
        (dropout): Dropout(p=0)
        (activation): GELU()
      )
      (input_sublayer): Sub

In [12]:
# -b 16 -e 30 --hidden 256 -l 7 --n_head 4 --lr 1e-4 --lr-decay 5 --final-lr 0.01
model = BERT(len(vocab), hidden=256, n_layers=8, attn_heads=8, dropout=0)
model.load_state_dict(torch.load('../result/chembl/ep00_it008000.pkl'))
model.eval()

BERT(
  (embedding): BERTEmbedding(
    (token): TokenEmbedding(45, 256, padding_idx=0)
    (position): PositionalEmbedding()
    (segment): SegmentEmbedding(3, 256, padding_idx=0)
    (dropout): Dropout(p=0)
  )
  (transformer_blocks): ModuleList(
    (0): TransformerBlock(
      (attention): MultiHeadedAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): Linear(in_features=256, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (output_linear): Linear(in_features=256, out_features=256, bias=True)
        (attention): Attention()
        (dropout): Dropout(p=0)
      )
      (feed_forward): PositionwiseFeedForward(
        (w_1): Linear(in_features=256, out_features=1024, bias=True)
        (w_2): Linear(in_features=1024, out_features=256, bias=True)
        (dropout): Dropout(p=0)
        (activation): GELU()
      )
      (input_sublayer): Sub

In [13]:
st,ed = 0,100
X_train = model.encode(xid_train[st:ed], xseg_train[st:ed]).detach().numpy()
while ed<len(xid_train):
    st += 100
    ed += 100
    X_train = np.vstack([X_train, model.encode(xid_train[st:ed], xseg_train[st:ed]).detach().numpy()])

In [14]:
st,ed = 0,100
X_test = model.encode(xid_test[st:ed], xseg_test[st:ed]).detach().numpy()
while ed<len(xid_test):
    st += 100
    ed += 100
    X_test = np.vstack([X_test, model.encode(xid_test[st:ed], xseg_test[st:ed]).detach().numpy()])

In [15]:
X_train.shape

(1161, 220, 1024)

In [16]:
# X_train = np.mean(X_train, axis=1)
# X_test = np.mean(X_test, axis=1)
X_train = X_train[:,0,:]
X_test = X_test[:,0,:]
#y_train = df_train['solubility']
#y_test = df_test['solubility']

# Prediction
### MLP

In [17]:
# Default
n = 10
r2 = np.zeros(n)
rmse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor()
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    rmse[i] = mean_squared_error(y_test, y_pred)**0.5

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test RMSE: {:.4f} ± {:.4f}".format(np.mean(rmse), np.std(rmse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Test R2: 0.8271 ± 0.0154
Test RMSE: 0.7799 ± 0.0347


In [18]:
def objective_mlp(trial):
    n_layers = trial.suggest_int('n_layers', 1,3)
    layers = []
    for i in range(n_layers):
        n_units = int(trial.suggest_loguniform('n_units_l{}'.format(i), 1, 1000))
        layers.append(n_units)
    
    n_folds = 4
    score = 0
    for _ in range(n_folds):
        mlp = MLPRegressor(hidden_layer_sizes=layers)
        X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train)
        mlp.fit(X_trn, y_trn)
        y_pred = mlp.predict(X_val)
        score += mean_squared_error(y_val, y_pred)**0.5
    return score/n_folds

study = optuna.create_study()
study.optimize(objective_mlp, n_trials=100)

[I 2019-04-05 19:43:33,687] Finished a trial resulted in value: 0.8117687827058929. Current best value is 0.8117687827058929 with parameters: {'n_layers': 1, 'n_units_l0': 676.913989129203}.
[I 2019-04-05 19:43:52,155] Finished a trial resulted in value: 0.7903470378045725. Current best value is 0.7903470378045725 with parameters: {'n_layers': 2, 'n_units_l0': 18.761422262657586, 'n_units_l1': 555.2435318186116}.
[I 2019-04-05 19:45:06,264] Finished a trial resulted in value: 0.7562492455982899. Current best value is 0.7562492455982899 with parameters: {'n_layers': 1, 'n_units_l0': 537.9322034028806}.
[I 2019-04-05 19:46:09,841] Finished a trial resulted in value: 0.7917897656328867. Current best value is 0.7562492455982899 with parameters: {'n_layers': 1, 'n_units_l0': 537.9322034028806}.
[I 2019-04-05 19:46:50,297] Finished a trial resulted in value: 0.847726537672051. Current best value is 0.7562492455982899 with parameters: {'n_layers': 1, 'n_units_l0': 537.9322034028806}.
/home/ho

[I 2019-04-05 19:51:41,263] Finished a trial resulted in value: 0.8282462561224678. Current best value is 0.7423980968895423 with parameters: {'n_layers': 1, 'n_units_l0': 140.68067592782435}.
[I 2019-04-05 19:53:42,964] Finished a trial resulted in value: 0.7532499585935393. Current best value is 0.7423980968895423 with parameters: {'n_layers': 1, 'n_units_l0': 140.68067592782435}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-04-05 19:53:57,491] Finished a trial resulted in value: 0.7605218677053509. Current be

[I 2019-04-05 20:09:31,246] Finished a trial resulted in value: 0.7845244531617999. Current best value is 0.7423980968895423 with parameters: {'n_layers': 1, 'n_units_l0': 140.68067592782435}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/p

[I 2019-04-05 20:15:18,420] Finished a trial resulted in value: 0.8081841366838136. Current best value is 0.7307546024704381 with parameters: {'n_layers': 2, 'n_units_l0': 21.833736591240573, 'n_units_l1': 866.9707915950405}.
[I 2019-04-05 20:16:19,810] Finished a trial resulted in value: 0.777186831945964. Current best value is 0.7307546024704381 with parameters: {'n_layers': 2, 'n_units_l0': 21.833736591240573, 'n_units_l1': 866.9707915950405}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-04-05 20:16:33,524] F

[I 2019-04-05 20:27:44,754] Finished a trial resulted in value: 0.8258315031667702. Current best value is 0.7307546024704381 with parameters: {'n_layers': 2, 'n_units_l0': 21.833736591240573, 'n_units_l1': 866.9707915950405}.
[I 2019-04-05 20:28:17,659] Finished a trial resulted in value: 0.7212893106085762. Current best value is 0.7212893106085762 with parameters: {'n_layers': 3, 'n_units_l0': 128.87970317783004, 'n_units_l1': 337.82722501506765, 'n_units_l2': 142.52646094526168}.
[I 2019-04-05 20:28:52,104] Finished a trial resulted in value: 0.7462260738358104. Current best value is 0.7212893106085762 with parameters: {'n_layers': 3, 'n_units_l0': 128.87970317783004, 'n_units_l1': 337.82722501506765, 'n_units_l2': 142.52646094526168}.
[I 2019-04-05 20:31:01,839] Finished a trial resulted in value: 0.7530667570250678. Current best value is 0.7212893106085762 with parameters: {'n_layers': 3, 'n_units_l0': 128.87970317783004, 'n_units_l1': 337.82722501506765, 'n_units_l2': 142.52646094

In [22]:
# Optimized

n = 10
r2 = np.zeros(n)
rmse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor((129,338,143))
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    rmse[i] = mean_squared_error(y_test, y_pred)**0.5

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test RMSE: {:.4f} ± {:.4f}".format(np.mean(rmse), np.std(rmse)))

Test R2: 0.8291 ± 0.0217
Test RMSE: 0.7747 ± 0.0488


### RF

In [62]:
#Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor()
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-package

Test R2: 0.7754 ± 0.0095
Test MSE: 0.9171 ± 0.0387


In [45]:
def objective_rf(trial):
    max_depth = int(trial.suggest_loguniform('max_depth', 2, 100))
    n_estimators = int(trial.suggest_loguniform('n_estimators', 2, 1000))
    max_features = trial.suggest_int('max_features', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        
    n_folds = 4
    score = 0
    for _ in range(n_folds):
        rf = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features,
                              min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train)
        rf.fit(X_trn, y_trn)
        y_pred = rf.predict(X_val)
        score += mean_squared_error(y_val, y_pred)
    return score/n_folds

study = optuna.create_study()
study.optimize(objective_rf, n_trials=100)

[I 2019-03-27 16:00:50,059] Finished a trial resulted in value: 1.5218836980010286. Current best value is 1.5218836980010286 with parameters: {'max_depth': 15.380039789241337, 'n_estimators': 147.90052040104513, 'max_features': 3, 'min_samples_split': 8, 'min_samples_leaf': 9}.
[I 2019-03-27 16:00:50,973] Finished a trial resulted in value: 1.4251916881928215. Current best value is 1.4251916881928215 with parameters: {'max_depth': 21.895441401945902, 'n_estimators': 88.38222724853792, 'max_features': 4, 'min_samples_split': 7, 'min_samples_leaf': 8}.
[I 2019-03-27 16:00:51,085] Finished a trial resulted in value: 1.348715189557296. Current best value is 1.348715189557296 with parameters: {'max_depth': 90.10057829792703, 'n_estimators': 7.645999324433651, 'max_features': 6, 'min_samples_split': 9, 'min_samples_leaf': 8}.
[I 2019-03-27 16:00:51,581] Finished a trial resulted in value: 1.8121345064864918. Current best value is 1.348715189557296 with parameters: {'max_depth': 90.1005782979

[I 2019-03-27 16:03:00,300] Finished a trial resulted in value: 1.2987897176560308. Current best value is 1.0300176525975921 with parameters: {'max_depth': 26.27709115303057, 'n_estimators': 40.736098033085035, 'max_features': 5, 'min_samples_split': 6, 'min_samples_leaf': 4}.
[I 2019-03-27 16:03:00,637] Finished a trial resulted in value: 1.3344446829133374. Current best value is 1.0300176525975921 with parameters: {'max_depth': 26.27709115303057, 'n_estimators': 40.736098033085035, 'max_features': 5, 'min_samples_split': 6, 'min_samples_leaf': 4}.
[I 2019-03-27 16:03:09,057] Finished a trial resulted in value: 1.1569104959044276. Current best value is 1.0300176525975921 with parameters: {'max_depth': 26.27709115303057, 'n_estimators': 40.736098033085035, 'max_features': 5, 'min_samples_split': 6, 'min_samples_leaf': 4}.
[I 2019-03-27 16:03:10,472] Finished a trial resulted in value: 1.1966306525547787. Current best value is 1.0300176525975921 with parameters: {'max_depth': 26.2770911

In [47]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor(max_depth=26, n_estimators=41, max_features=5,
                              min_samples_split=6, min_samples_leaf=4)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

Test R2: 0.7342 ± 0.0045
Test MSE: 1.0853 ± 0.0185
